In [1]:
import pandas as pd
import numpy as np
import pyodbc
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from requests import get

In [ ]:
import getpass
#conexão com o dataWarehouse
server = 'localhost' 
database = 'dataWarehouse' 
username = '' 
password =  getpass.getpass()
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()


In [467]:
#carrega a dim_cliente
query = "SELECT * from dim_cliente where tipo_cliente = 'J' and mercado_cliente = 'I' and CPF_CNPJ > 9999999999 and situacao_cliente = 'A';"
queryDC = "SELECT * from dim_cliente;"
df = pd.read_sql(query, cnxn)
dim_cliente = pd.read_sql(queryDC, cnxn)
df.shape, dim_cliente.shape

((13790, 12), (18223, 12))

In [272]:
#função para scraping de CNPJ
def scrapNome(cnpj):
    url = 'https://www.consultascnpj.com/matriz/' + cnpj[:8]
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    }
    response = get(url, headers = headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    nome = html_soup.find_all('h1')
    if not nome:
        return scrapNome2(cnpj)
    else: 
        print(html_soup.h1.text.upper())
        return html_soup.h1.text.upper()
def scrapNome2(cnpj):
    url = 'https://www.cnpj.pro/info/' + cnpj
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    }
    response = get(url, headers = headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    nome = html_soup.find_all('h1', class_ = 'h2')
    if not nome:
        scrapNome3(cnpj)
    else:
        print(nome[0]).text.upper()
        return nome[0].text.upper()
def scrapNome3(cnpj):
    url = 'https://www.foneempresas.com/telefone/cnpj/cibra/' + cnpj
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    }
    response = get(url, headers = headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    type(html_soup)
    nome = html_soup.find_all('span', class_ = 'text')
    #print(nome[1].text)
    if not nome:
        print(cnpj + ' não encontrado')
        return ''
    else:
        print(nome[1]).text.upper()
        return nome[1].text


In [16]:
df.dtypes

cod_cliente          int64
nome_cliente        object
apelido_cliente     object
cidade_cliente      object
UF_cliente          object
pais_cliente        object
endereco_cliente    object
situacao_cliente    object
mercado_cliente     object
end_completo_cli    object
CPF_CNPJ             int64
tipo_cliente        object
dtype: object

In [304]:
dfGrupo.shape

(2391, 15)

In [312]:
####Carrega o prefixo do CNPJ em CNPJ e o sufixo em filial
df['CNPJ'] = df['CPF_CNPJ'].astype('object')
df['Filial'] = df['CNPJ'].apply(lambda x: str(x).zfill(14)).str[8:12]
df['CNPJ'] = df['CNPJ'].apply(lambda x: str(x).zfill(14)).str[:8]

In [294]:
## agrupa os cnpjs do mesmo grupo e cria a coluna C com a quantidade e filiais
df['C'] = df.groupby('CNPJ')['CNPJ'].transform('size')
df.sort_values(by=['C', 'CNPJ'], inplace=True)
## cria o dataframe grupo com todos os clientes que possuem mais de uma filial
dfGrupo = df.loc[df['C'] > 1]

In [320]:
#dataframe contagem apenas com os valores duplicados em df
contagem = df.loc[df['CNPJ'].duplicated(False)]



In [391]:
#agrupados por CNPJ os valores de contagem

contagem['CPF_CNPJ'] = contagem['CPF_CNPJ'].astype('object').apply(lambda x: str(x).zfill(14))
grupo = contagem.groupby('CNPJ').first().iloc[:,:0]


In [469]:
# Transforma CNPJ em object, Cria coluna CNPJ com o prefixo do CNPJ, faz left join entre dim_cliente e grupo de nomes

dim_cliente['CNPJ'] = dim_cliente['CPF_CNPJ'].astype('object')
dim_cliente['CNPJ'] = dim_cliente['CNPJ'].apply(lambda x: str(x).zfill(14)).str[:8]
dim_cliente = pd.merge(dim_cliente, nomeGrupo, left_on = 'CNPJ', right_on = 'CNPJ', how = 'left')
dim_cliente.shape

(18223, 14)

In [284]:
#backup dos nomes
nome.to_csv('nomes.csv')

In [476]:
for i, row in dim_cliente2.iterrows():
     cursor.execute("UPDATE dim_cliente2 set nome_grupo = ? where cod_cliente = ?", row.Nome, row.cod_cliente)
cnxn.commit()
cursor.close()